# find tracerRNA

of FLJLKKHA_00012
problem: ha 2 active working cas loci. e il tracr mi sa che l'ha trovato in tutti e due, ma solo uno è segnato.
l'altro (greppato a mano) è:

- Cas9 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     15614   18553   .       -       0       ID=FLJLKKHA_00012;Parent=FLJLKKHA_00012_gene;eC_number=3.1.-.-;Name=cas9_1;gene=cas9_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:Q9CLT2;locus_tag=FLJLKKHA_00012;product=CRISPR-associated endonuclease Cas9;protein_id=gnl|X|FLJLKKHA_00012
- Cas1 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     14715   15614   .       -       0       ID=FLJLKKHA_00011;Parent=FLJLKKHA_00011_gene;eC_number=3.1.-.-;Name=cas1_1;gene=cas1_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:G3ECR2;locus_tag=FLJLKKHA_00011;product=CRISPR-associated endonuclease Cas1;protein_id=gnl|X|FLJLKKHA_00011
- Cas2 : NODE_49_length_176785_cov_12.2725       Prodigal:2.6    CDS     14378   14707   .       -       0       ID=FLJLKKHA_00010;Parent=FLJLKKHA_00010_gene;eC_number=3.1.-.-;Name=cas2_1;gene=cas2_1;inference=ab initio prediction:Prodigal:2.6,similar to AA sequence:UniProtKB:G3ECR3;locus_tag=FLJLKKHA_00010;product=CRISPR-associated endoribonuclease Cas2;protein_id=gnl|X|FLJLKKHA_00010

mentre le Cas1,2,9 segnate giu, appartengono al contig:

NODE_56_length_167430_cov_15.277


In [2]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [3]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [4]:
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)
tabellazza.columns

Index(['Unnamed: 0.1', 'Genome Name', 'Study', 'Sample Name', 'Assembler',
       'Genome Size', 'N50', '# Contigs', 'Completeness', 'Contamination',
       'Strain Heterogeneity', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_CRISPR', 'prokka_cas', 'uniref_cas'],
      dtype='object')

In [5]:
seqid ="FLJLKKHA_00012"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  FLJLKKHA_00012
NODE_49_length_176785_cov_12.2725
BritoIL_2016
BritoIL_2016__M1.29.ST__bin.30
979
[('.', '13625', '14320')]
----------------------------------------------------------------------


In [23]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

######################################################################TODO THIS MUST BE INSIDE filename_discrepancies
# take into account filename discrepancies
if dataset.startswith("ZeeviD"):  #TODO guarda se funziona quando finisci di girare minced_runner
         s3_dataset="ZeeviD_2015"
         #genomename=filename_discrepancies.change_to_megahit(genomename)
         s3_genomename=genomename
         try:
             genomename=genomename.replace("ZeeviD_2015", "ZeeviD_2015_B")
             prokkafile_ofsample=open(annodir+"/justminced/ZeeviD_2015_B/"+genomename+".crisprcas.gff.minced")
             prokkafile_ofsample.close()
             dataset="ZeeviD_2015_B"
         except:
             genomename=genomename.replace("ZeeviD_2015_B", "ZeeviD_2015_A")
             dataset="ZeeviD_2015_A"
else:
    s3_dataset=dataset
    s3_genomename=genomename
    dataset=filename_discrepancies.s3(s3_dataset,r=True) # get working dataset name from s3 dataset name
    genomename=genomename.replace(s3_dataset, dataset)

if filename_discrepancies.dataset_has_megahit(dataset,genomename):
    genomename=filename_discrepancies.change_to_megahit(genomename)

samplename = genomename.split("__")[1]
########################################################################

# get CRISPR spacer and repeat sequence

mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")
for line in f.readlines():
    print(line)
    if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
    line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

        repeats.append(line.split("\t")[2])

        repeat_start_pos.append(line.split("\t")[0])

        if not line.split("\t")[3]=="\n":
            spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai genomename della cas e/o seqid della cas (anzi famo solo seqid, che magari ci sn doppioni in genomi) e ti dà tt il locus

# print start and stop of CRISPR
unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
crispr_start, crispr_stop = int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\""))

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.

prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

prokkaslist=prokka_anno.split(">")
prokkaslist.pop(0)
# get contigname of current effectorcas   
cas_position={}
for casanno in prokkaslist:
    unostranosplit=casanno.split("Name=")
    casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
    unaltrostranosplit=casanno.split("\t")
    cas_start=unaltrostranosplit[3]
    cas_stop=unaltrostranosplit[4]
    cas_position["Cas"+casnumber]=[int(cas_start),int(cas_stop)]

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", crispr_start, crispr_stop)
        print("Cas9 si trova", cas_position["Cas9"])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        cas9start=[int(x) for x in cas_position["Cas9"]][0]
        cas9stop=[int(x) for x in cas_position["Cas9"]][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)

        #extact a locus, 200 bp a monte e a valle dei cosi che ho:
        starts, stops=[startstop[0]for startstop in cas_position.values()], [startstop[1]for startstop in cas_position.values() ]
        minimo=min([crispr_start]+starts)
        print("start: ", minimo)
        massimo=max([crispr_stop]+stops)
        print("end: ", massimo)

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	FLJLKKHA_00012
SGB:	9311
Genome Name:	 BritoIL_2016__M1.29.ST__bin.30
Contig:	NODE_49_length_176785_cov_12.2725 
-----------------------------------------------------

Sequence 'NODE_49_length_176785_cov_12.2725' (176785 bp)



CRISPR 1   Range: 13625 - 14320

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

13625		GTTATCGCTTCCTTTCCAGCCTAAACGCGGTATAAT	GCGGGTTGGTCCTTCGGCGCTTCGCTCTCT	[ 36, 30 ]

13691		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	CGGCCTCGGCAACAGATACACCGTCAACCT	[ 36, 30 ]

13757		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	ACCTTGCCCTTTGTTTGTTTTGGAAATCGT	[ 36, 30 ]

13823		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	TCGTTCTTCAACCCCTGCAACCCATACTGA	[ 36, 30 ]

13889		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	CTGCCACCGCGCCGAAAACACCCGCCGCAA	[ 36, 30 ]

13955		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	CGGCGTAAATGCGTTTTGACGCGTATCCGT	[ 36, 30 ]

14021		GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT	CTAT

In [7]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['13625', '13691', '13757', '13823', '13889', '13955', '14021', '14087', '14153', '14219', '14285', '102979', '103045', '103111', '103177', '103243', '103309', '103375', '103441', '103507', '103573', '103639', '103705', '103771', '103836', '103902', '103969', '104035', '104101', '104167', '104233', '104299', '104365', '104431', '104497', '104563', '104629', '104695', '104761', '104827', '104895', '104961', '105027', '105093', '105159', '105225'] :
-----------------------------------------------------

rpt2|NODE_49_length_176785_cov_12.2725|BritoIL_2016__M1.29.ST__bin.30|FLJLKKHA_00012	NODE_49_length_176785_cov_12.2725	96.30	36	27	1	0	TTATCGCTTCCTTTCCAATCAAAACGC	TTATCGCTTCCTTTCCAATTAAAACGC	18691	18665	8e-08	minus

rpt3|NODE_49_length_176785_cov_12.2725|BritoIL_2016__M1.29.ST__bin.30|FLJLKKHA_00012	NODE_49_length_176785_cov_12.2725	100.00	36	

## extract sequences

In [22]:
# attenzioe: strano: poiché è sul minus strand, quindi start e stop sono al contrario. ma secondo i miei calcoli, per come è fatto blast, dovrebbe hittare sul plus strand..
#qst vuol dire che sul - c'è la repeat,ee onn lantirepeat. quindi funge male....
# che è testimoniato piu a valle dal fatto che trovo n  terminatore la  vicino, ma lo trovo dopo...
#omg che sia un tracrRNA NON trans?
# D:

# questo potrebbe voler dire che il crispr arraay è in realtà sullo strand opposto?

matched_rep_start=18675
matched_rep_stop=18691

In [9]:
# this is the sequence matching the repeat:
matched_rep=Seq("TTATCGCTTCCTTTCCAATTAAAACGC")

#rpts which match:
rpt2=Seq("GTTATCGCTTCCTTTCCAATCAAAACGCGGTATAAT")

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

GCGTTTTAATTGGAAAGGAAGCGATAA


27

In [11]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("1"):
        print(line)
f.close()

  1464  Rnamotif - GCTCTTTCACTCCCCTTTCGGGGTTCTTTTCACCT -6.40

 10200  Rnamotif + CATTTATTCCATGGGGTTGACGGTCGGTCCCAaTATCTTTTCCAG -6.90

 10569  Rnamotif - AAACCCTCCATGTGGGGATTTTCCTCCGCcTTTTGGTATTCG -7.70

 10731  Rnamotif + CGAACACACGACGTTGCAGGTCGGCGACGTTTTGCTGCTGT -7.80

 13902      Both - GGTGTTTTCGGCGCGGTGGCAGATTATACCGCGTTTTGATTGGAA -11.30

 14322      Both - AAAAGAAAAAACCCTTGTTTTTCAAGGGcTTTTTCGTTGGC -8.70

 14324      Both + CAACGAAAAAGCCCTTGAAAAACAAGGGTTTTTTCTTTTA -8.20

 14377  Rnamotif - AACGGAAAAAATCGGCGCGGAACAGCTGGTTTTATTGTGAT -5.30

 14855  Rnamotif - AAGACAAACAACGGCTGGTTGCCGTTTTGACGCGCA -6.60

 15010      Both - CGCAGTTTGGTCGGGGCGGGGCTGTTGCCTTGTTTCGGTTTGCA -8.70

 18892      Both - CAATTCTTAAAGCCTTGAAGTCCGCCGCTTCAAGGCtTTTTCCGTATCAA -10.20

 18895      Both + ATACGGAAAAAGCCTTGAAGCGGCGGACTTCAAGGCTTTAAGAATTGGT -10.20

 19135  Rnamotif - AATCAACAACGGAACGGCAGATTTTCCGTTCTTTTGCAGCAGA -7.20

100013  Rnamotif + GTTGCCTGTCGCTTCGTCGCGCAGAGCGGGGTTTTTCAAAACG -4.80

101037  Rnamotif + AATGT

In [12]:
# this is the rho-independent trnascription terminator, going from 14719 or 14722 to bo (+1 perché conti il primo tipo), 

terminator_end=18895 -1 # qui è sbagliato, mavabbe proviamo
terminator=Seq("CAATTCTTAAAGCCTTGAAGTCCGCCGCTTCAAGGCtTTTTCCGTATCAA".upper())
print("terminator length: ", len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

terminator length:  50


In [21]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
#solo per lui:
contigname="NODE_49_length_176785_cov_12.2725"
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq
print(len(contigseq[matched_rep_start-1:matched_rep_stop]))
#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb
print(contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement())
# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)


pre_tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

0
test repeat:  GCGTTTTAATTGGAAAGGAAGCGATAA 27 False

test terminator:  CAATTCTTAAAGCCTTGAAGTCCGCCGCTTCAAGGCTTTTTCCGTATCAA 50 False
pre tracrRNA:  0
mario sequence:  0


ora ha una lunghezza simile, e finisce in modo simile a quello di _S. pyogenes_  TTTTTTTaltre

In [47]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt3, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


ATTGTAGTT--CCCTAATTT-TTCTTGGTATGTTATAAT
 |||||| |  ||| || || ||||||||||| | | ||
-TTGTAG-TAACCC-AA-TTGTTCTTGGTATG-T-T-AT
  Score=29

rpt length: 36


In [51]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx( rpt3,(antirepeat+mario[:4]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

A-TT---GTAGTT--CCCTAATTT-TTCTTGGTATGTTATAAT
  ||   |||| |  ||| || || ||||||||||| | | ||
-CTTTTTGTAG-TAACCC-AA-TTGTTCTTGGTATG-T-T-AT
  Score=29



In [ ]:
#no non fa nulla quel CTTT in più

In [54]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id CJFLGLGO_01367
----------------------------------------------------------------------

- contig name: NODE_137_length_63930_cov_6.02239
- Dataset:  Obregon-TitoAJ_2015
- Bin:  Obregon-TitoAJ_2015__SM40__bin.4
- SGB:  4329
----------------------------------------------------------------------

- Length:  91 
- position:  49289 49380
TTATAACATACCAAGAACAATTGGGTTACTACAAAAAGGTAGAAAACCGAAAAGCTCTGACGGCTCCTTTTTGGAGCCGTTATCTTTTTCT
